In [2]:
import torch
import torch.optim as optim
import torch.autograd as autograd
import autograd.numpy as np
from tqdm import tqdm, trange
from kernel import RBF
from Tmy_svgd import tmySVGD

from svgd import SVGD
import argparse
from mysvgd import mySVGD
from utils import plot_particles
import torch.distributions as D
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

ModuleNotFoundError: No module named 'src'

In [ ]:
parser = argparse.ArgumentParser(description='Running xshaped experiment.')

parser.add_argument('--dim', type=int, default=9, help='dimension')

parser.add_argument('--effdim', type=int, default=-1, help='dimension')
parser.add_argument('--lr', type=float, default=0.01, help='learning rate')
parser.add_argument('--delta', type=float,default=0.01,help='stepsize for projections')
parser.add_argument('--T', type=float, default=1e-4, help='noise multiplier for projections')
parser.add_argument('--lr_g', type=float, default=0.1, help='learning rate for g')
parser.add_argument('--nparticles', type=int,default=50, help='no. of particles')
parser.add_argument('--epochs', type=int, default=20000, help='no. of epochs')
parser.add_argument('--metric', type=str, default="energy", help='distance metric')
parser.add_argument('--noise', type=str, default="True", help='whether to add noise')
parser.add_argument('--kernel', type=str, default="rbf", help='kernel')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--seed', type=int, default=235, help='random seed')
parser.add_argument('--suffix', type=str, default="", help='suffix for res folder')
parser.add_argument('--m', type=int, help='no. of projections')
parser.add_argument('--save_every', type=int, default=200, help='step intervals to save particles')
parser.add_argument('--method', type=str, default="all", help='which method to use')

In [ ]:
args = parser.parse_args([])
dim = args.dim
lr = args.lr
lr_gsvgd = args.lr
delta = args.delta
T = args.T
nparticles = args.nparticles
epochs = args.epochs
seed = args.seed
eff_dims = [args.effdim] if args.effdim > 0 else [1, 2, 5]
save_every = args.save_every # save metric values
print(f"Running for dim: {dim}, lr: {lr}, nparticles: {nparticles}")

device = torch.device(f'cuda:{args.gpu}' if args.gpu != -1 else 'cpu')

metric = args.metric
Kernel = RBF
kernel = Kernel(method="med_heuristic")

In [ ]:
print(f"Device: {device}")
torch.manual_seed(seed)

## target density
means = torch.zeros(dim, device=device)

torch.manual_seed(0)
'''A
A = torch.randn(dim,dim).to('cuda') * 0.9
A = torch.matmul(A, A.T)

m = torch.max(A) 
B = torch.eye(dim).to('cuda') * m + 0.1
diag = torch.diag(A)
cov = A + B'''

cov = torch.eye(dim, device=device)

distribution = D.MultivariateNormal(means.to(device), cov)

# sample from target (for computing metric)
x_target = distribution.sample((nparticles, ))
# sample from variational density
torch.manual_seed(235)
x_init = 2 + np.sqrt(2) * torch.randn(nparticles, *distribution.event_shape, device=device)


In [ ]:
## SVGD
Kernel = RBF
if args.method in ["SVGD", "all"]:
    print("Running SVGD")
    # sample from variational density
    x = x_init.clone().to(device)
    kernel = Kernel(method="med_heuristic")
    svgd = SVGD(distribution, kernel, optim.Adam([x], lr=lr), device=device)
   
    svgd.fit(x, epochs, verbose=True, save_every=save_every)
    

    # plot particles
    fig_svgd = plot_particles(
        x_init.detach(), 
        x.detach(), 
        distribution, 
        d=6.0, 
        step=0.1, 
        concat=means[2:]
    )

In [ ]:
theta  = x
index_svgd = []
samn_svgd = []
for i in range(theta.shape[0]):
    samn_svgd.append(torch.linalg.norm(theta[i,:].cpu()).item())
    index_svgd.append(i)

import matplotlib.pyplot as plt
plt.scatter(index_svgd, samn_svgd, c='blue')
cov_svgd = torch.cov(theta.T)
print(torch.linalg.norm(cov - cov_svgd))
print(cov_svgd)
print(cov - cov_svgd)

In [ ]:
print("Running tmySVGD")

def score(X):
        X_cp = X.clone().detach().requires_grad_()
        log_prob = distribution.log_prob(X_cp)
        score_func = autograd.grad(log_prob.sum(), X_cp)[0]
        return score_func


# sample from variational density
res = []
rres = []
steps = []
lr = 0.01
x0 = torch.randn(nparticles, dim)
vector1  = torch.randn(nparticles, dim).to('cuda')

for i in trange(20):
    
    theta, vector = tmySVGD(kernel).update(x0, score,  k = 2, n_iter = 50,  debug = False, lr= lr, vector=vector1)
    #mean = np.mean(theta, axis=0)  + np.random.random(1)
    #var_theta = np.cov(theta.T) + np.random.random(1)
    #x0 = np.random.multivariate_normal(mean, var_theta,num)
    x0 = theta
    vector1 = vector
    
    # res.append(np.linalg.norm(theta)/D)
    
    mean_svgd = torch.mean(theta, axis=0)

    var_svgd = torch.cov(theta.T) 
    res.append(torch.linalg.norm(mean_svgd).item())
    rres.append(torch.linalg.norm(var_svgd).item()) 
    if i % 20 == 0:
        print(theta.shape)
        print(torch.linalg.norm(cov- var_svgd) )
        print(torch.linalg.norm(mean_svgd - means))
    
leng = len(res)
x = np.arange(1,leng+1)
plt.figure(1)
plt.plot(x,res)
plt.figure(2)
plt.plot(x,rres)

In [ ]:

fig_svgd = plot_particles(
        x_init.detach(), 
        theta.detach(), 
        distribution, 
        d=6.0, 
        step=0.1, 
        concat=means[2:]
    )

In [ ]:
print("Running mySVGD")

def score(X):
        X_cp = X.clone().detach().requires_grad_()
        log_prob = distribution.log_prob(X_cp)
        score_func = autograd.grad(log_prob.sum(), X_cp)[0]
        return score_func


# sample from variational density
res = []
rres = []
steps = []
lr = 0.01
x0 = x_init
vector1  = torch.randn(nparticles, dim).to(device)

for i in trange(25):
    
    theta, vector = mySVGD().update(x0, score,  k = 2, n_iter = 50,  debug = False, lr= lr, vector=vector1, device=device)
    #mean = np.mean(theta, axis=0)  + np.random.random(1)
    #var_theta = np.cov(theta.T) + np.random.random(1)
    #x0 = np.random.multivariate_normal(mean, var_theta,num)
    x0 = theta
    vector1 = vector
    
    # res.append(np.linalg.norm(theta)/D)
    
    mean_svgd = torch.mean(theta, axis=0)

    var_svgd = torch.cov(theta.T) 
    res.append(torch.linalg.norm(mean_svgd).item())
    rres.append(torch.linalg.norm(var_svgd).item()) 
    if i % 20 == 0:
        
        print(torch.linalg.norm(cov- var_svgd) )
        print(torch.linalg.norm(mean_svgd - means))
    
leng = len(res)
x = np.arange(1,leng+1)
plt.figure(1)
plt.plot(x,res)
plt.figure(2)
plt.plot(x,rres)

In [ ]:

fig_svgd = plot_particles(
        x_init.detach(), 
        theta.detach(), 
        distribution, 
        d=6.0, 
        step=0.1, 
        concat=means[2:]
    )